In [2]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm
import swifter
import json
tqdm.pandas()

In [ ]:
df = pd.read_csv('data/995,000_row_cleaned.csv')

In [3]:
test_df = pd.read_csv('data/news_sample_6.csv')

In [4]:
# Remove columns 
test_df = test_df[['title', 'content', 'source', 'publishedAt', 'url', 'urlToImage']]

KeyError: "['source', 'publishedAt', 'urlToImage'] not in index"

In [ ]:
vocab = {}
def create_vocab(lst):
    lst = ast.literal_eval(lst)
    for word in lst:
        if word in vocab:
            vocab[word] += 1
        else:
            vocab[word] = 1

df['content'].swifter.progress_bar(True).apply(create_vocab)
print(vocab)

In [ ]:
shortend_vocab = dict(sorted(vocab.items(), key=lambda item: item[1], reverse=True)[:10000])
with open('10000vocab.json', 'w') as f:
    json.dump(shortend_vocab, f)

In [ ]:
def count_freq(lst):
    dic = {key: 0 for key in list(vocab)}
    lst = ast.literal_eval(lst)
    for word in lst:
        if word in dic:
            dic[word] += 1
    return [value for value in dic.values()]

test_df['content'] = test_df['content'].swifter.progress_bar(True).apply(count_freq)

# Save cleaned data
output_path = 'data/995,000_row_counted.csv'
test_df.to_csv(output_path, index=False)
print(f"Fully cleaned; data saved to {output_path}")

In [ ]:
label_map = {"fake": 1, "satire":1, "conspiracy": 1, "unreliable": 1, "bias": 1, "rumor": 1, "junksci": 1, "hate": 1,
             "reliable": 0,  "clickbait": 0,   "political": 0}
labels = np.array([label_map.get(row['type']) for _,row in test_df.iterrows()])
features = np.array([count_freq(row['content']) for _,row in test_df.iterrows()])
dataset = np.column_stack((labels, features))

# Print shapes
print("Features shape:", features.shape)  # (num_samples, num_features)
print("Labels shape:", labels.shape)  # (num_samples,)

# Example output
print("First sample:", dataset[-1])